# Seasons by Aritzia

## 1. Skin Detection from Image

In [ ]:
from numpy.core.memmap import uint8
import cv2
import numpy as np

min_range = np.array([0,133,77]np.uint8)
max_range = np.array([235,173,127],np.uint8)
image = cv2.imread("image_50.png")

In [ ]:
YCRimage = cv2.cvtColor(image,cv2.COLOR_BGR2YCR_CB)
skin_area = cv2.inRange(YCRimage,min_range,max_range)
detected_skin = cv2.bitwise_and(image, image, mask = skin_area)

In [ ]:
cv2.imwrite("C:\\Users\\sadie\\Pictures\\detectedImage.png", 
            np.hstack([image,detected_skin]))

True

## 2. Face Detection from Webcam

In [ ]:
import cv2
import sys
cascPath = sys.argv[1]
faceCascade = cv2.CascadeClassifier(cascPath)
video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.cv.CV_HAAR_SCALE_IMAGE
    )

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

## 3. Skin Detection from Webcam

We modified the code from https://github.com/SamSamhuns/color_matching_and_skin_detection to increase both our and the judges' understanding of the product's intent. Code was modified instead of completed from scratch given our time and experience constraints.

### a. Skin Detection following Face Detection in Webcam

In [2]:
import numpy as np
import argparse
import cv2

def detect_and_display_skin(vid_src):
    cap = cv2.VideoCapture(vid_src)

    min_YCrCb = np.array([0, 133, 77], np.uint8)
    max_YCrCb = np.array([235, 173, 127], np.uint8)
    while(True):
        _, frame = cap.read()
        frame = cv2.flip(frame, 1)

        YCrCb_converted = cv2.cvtColor(frame, cv2.COLOR_BGR2YCR_CB)
        skinMask = cv2.inRange(YCrCb_converted, min_YCrCb, max_YCrCb)

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
        skinMask = cv2.erode(skinMask, kernel, iterations=2)
        skinMask = cv2.dilate(skinMask, kernel, iterations=2)
        skinMask = cv2.GaussianBlur(skinMask, (3, 3), 0)
        skin = cv2.bitwise_and(frame, frame, mask=skinMask)

        skin[np.where((skin != [0, 0, 0]).all(axis=2))] = [255, 255, 255]
        cv2.imshow('orig', frame)
        cv2.imshow('detected', skin)
        cv2.moveWindow('detected', skin.shape[1], 0)

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

### b. Skin Tone Detection by Color Matching with Thresholds

In [3]:
import cv2
import argparse
import numpy as np


def nothing(x):
    pass

def hsv_threshold_picker(img_path, wait_time=33, mask_val=255):
    """
    params:
        img_src: path to image
        wait_time: cv2.waitKey time, set 30 for Osx
        mask_val: bit value of masked bits, 0=black, 255=white
    """
    image = cv2.imread(img_path)
    output = image

    cv2.namedWindow('image')

    cv2.createTrackbar('Hue Min', 'image', 0, 179, nothing)
    cv2.createTrackbar('Sat Min', 'image', 0, 255, nothing)
    cv2.createTrackbar('Val Min', 'image', 0, 255, nothing)

    cv2.createTrackbar('Hue Max', 'image', 0, 179, nothing)
    cv2.createTrackbar('Sat Max', 'image', 0, 255, nothing)
    cv2.createTrackbar('Val Max', 'image', 0, 255, nothing)

    cv2.setTrackbarPos('Hue Max', 'image', 179)
    cv2.setTrackbarPos('Sat Max', 'image', 255)
    cv2.setTrackbarPos('Val Max', 'image', 255)

    hMin = sMin = vMin = hMax = sMax = vMax = 0
    phMin = psMin = pvMin = phMax = psMax = pvMax = 0

    while(1):
        hMin = cv2.getTrackbarPos('Hue Min', 'image')
        sMin = cv2.getTrackbarPos('Sat Min', 'image')
        vMin = cv2.getTrackbarPos('Val Min', 'image')

        hMax = cv2.getTrackbarPos('Hue Max', 'image')
        sMax = cv2.getTrackbarPos('Sat Max', 'image')
        vMax = cv2.getTrackbarPos('Val Max', 'image')

        lower = np.array([hMin, sMin, vMin])
        upper = np.array([hMax, sMax, vMax])

        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower, upper)
        output = cv2.bitwise_and(image, image, mask=mask)

        output[output == 0] = mask_val

        cv2.imshow('image', output)
        cv2.imshow('original', image)

        if cv2.waitKey(wait_time) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

### c. Skin Tone Detection with Webcam

In [7]:
import numpy as np
import argparse
import cv2


def detect_color_in_hsv(video_src, min_HSV, max_HSV):
    """
    params:
        video_src: path to video or 0 for webcam
        min_HSV: Lower boundary of HSV color space
        max_HSV: Upper boundary of HSV color space
    """
    video_src = 0 if video_src is None else video_src
    cap = cv2.VideoCapture(video_src)

    while(True):
        _, frame = cap.read()
        frame = cv2.flip(frame, 1)

        HSV_converted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        color_mask = cv2.inRange(HSV_converted, min_HSV, max_HSV)

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
        color_mask = cv2.erode(color_mask, kernel, iterations=2)
        color_mask = cv2.dilate(color_mask, kernel, iterations=2)
        color_mask = cv2.GaussianBlur(color_mask, (3, 3), 0)
        det_color = cv2.bitwise_and(frame, frame, mask=color_mask)

        det_color[np.where((det_color != [0, 0, 0]).all(axis=2))] = [
            255, 255, 255]
        cv2.imshow('detected', det_color)
        cv2.imshow('orig', frame)
        cv2.moveWindow('detected', 0, det_color.shape[0] - 120)

        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

## 4. Using Machine Learning to Generate Color Palettes & Product Recommendations from Skin Color

In [11]:
import pandas as pd
import numpy as np

In [26]:
np.random.seed(1234)

In [12]:
training_data1 = pd.read_csv("training_data1.csv")
training_data2 = pd.read_csv("training_data2.csv")
training_data3 = pd.read_csv("training_data3.csv")

training_data1.head(10)

,image_name,skin_color
0,IMG_0001,e9d7cf
1,IMG_0002,ecd9cb
2,IMG_0003,e4dcd7
3,IMG_0004,f1dbce
4,IMG_0005,e6cbb7
5,IMG_0006,f1d9cd
6,IMG_0007,f0cdbb
7,IMG_0008,eedbd4
8,IMG_0009,e5cbbe
9,IMG_0010,ebcfce


In [13]:
training_data2.head(10)

,skin_color,season
0,e9d7cf,light_summer
1,ecd9cb,light_spring
2,e4dcd7,light_summer
3,f1dbce,light_spring
4,e6cbb7,dark autumn
5,f1d9cd,soft_summer
6,f0cdbb,true_winter
7,eedbd4,true_spring
8,e5cbbe,true_winter
9,ebcfce,soft_summer


In [14]:
training_data3.head(10)

,season,product_color_r
0,light_summer,cool
1,light_spring,cool
2,light_summer,cool
3,light_spring,cool
4,dark_autumn,dark_neutral
5,soft_summer,light_neutral
6,true_winter,deep
7,true_spring,pastel
8,true_winter,deep
9,soft_summer,light_neutral


In [17]:
testing_data = pd.read_csv("testing_data.csv")
testing_data.head(10)

,image_name,skin_color
0,IMG_0001,e7cfc7
1,IMG_0002,eccfc0
2,IMG_0003,f3d4c1
3,IMG_0004,f7ddc4
4,IMG_0005,dfbda9
5,IMG_0006,e3b3a1
6,IMG_0007,e4bea8
7,IMG_0008,e7bca0
8,IMG_0009,e5b59a
9,IMG_0010,d4a283


### a. Converting Hexadecimal Color Codes to Decimals for Ease

In [25]:
x = int("e9d7cf",16)
print(x)

15325135


Given time and experience constraints, we would ideally convert each hexadecimal color code in the training data to a decimal.

### b. Defining the 12 Seasons with Color Ranges

From the decimal outputs, we would then define the range of skin tones that make up each of the 12 seasons.

### c. Defining Product Recommendation Color Palettes

Based on the colors that Aritzia's clothing uses, we would define each color palette as a range of colors converted from hexadecimal codes into decimals.

### d. Build and Train Convolution Neural Networks

After defining all variables, we would go on to train a convolution neural network that would be able to classify a customer's personalized season based on their skin tone.

Another convolution neural network would also be needed to classify a customer's product recommendations using color palettes based on their personalized season.

Finally, we would use the final trained model to predict on the test dataset. We would also calculate both the in-sample and out-of-sample/validation accuracies. 